# Creating Standard Training Data

In this notebook we will be doing the following :
  1. Build & perform basic text cleaning operations/pipeline on the documents
  2. Convert the data in training data format. i.e. _label_tag1 _label_tag2
  3. Split the dataset into : train,dev and test sets  
  4.  Divide the dataset into Groups/Classes

    a. Dividing the entire Dataset(~1M) into 14 groups/class.

    b. Check the Label Distribution of Labels in each 14 groups/class.


  5. Create Corpus & Label Dictionary : Flair Corpus

**HOT TIP** : *Save them as pickle for easy rendering for experiments*



In [0]:
# First let's check what has Google given us ! Thank you Google for the GPU

!nvidia-smi

In [0]:
# Let's mount our G-Drive. Hey !! Because for GPU you now give your data to Google 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Install necessary packages and restart the environment

! pip install tiny-tokenizer
! pip install  flair

     |████████████████████████████████| 194kB 1.3MB/s 
     |████████████████████████████████| 389kB 4.2MB/s 
     |████████████████████████████████| 1.0MB 6.5MB/s 
     |████████████████████████████████| 798kB 7.4MB/s 
     |████████████████████████████████| 778kB 10.9MB/s 
     |████████████████████████████████| 1.0MB 13.3MB/s 
     |████████████████████████████████| 860kB 16.9MB/s 
     |████████████████████████████████| 348kB 19.3MB/s 
     |████████████████████████████████| 3.8MB 20.7MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=10334ec6e1f5ef05ef3434701a3676e8318febb2063b241ec6ae33d2c8313995
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=573bcbda8732f8ce230e2f0a3969374d55540a1ec975f24d0dfaa7b1d89ec1d0
  Stored in director

In [0]:
# Let's import our packages !

import pandas as pd
from tqdm import tqdm
import html
import re
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
# import flair
import pickle
from torch.optim.adam import Adam

# Making Corpus

from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [0]:
## Mentioning where is our data located on G-Drive. Make sure to rectify your path

path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/'
tag_group = '/content/drive/My Drive/ICDMAI_Tutorial/stack-overflow-tag-network/stack_network_nodes.csv'
data ='filtered_data/question_tag_text_mapping.pkl'

In [0]:
# Let's see the main Data-Set

question_tag = pd.read_pickle(path+data)
question_tag.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,CreationMonth,CreationYear,Tag
0,120,83.0,2008-08-01 15:50:08,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,8,2008,"[asp.net, sql]"
1,260,91.0,2008-08-01 23:22:08,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,8,2008,"[c#, .net]"
2,330,63.0,2008-08-02 02:51:36,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,8,2008,[c++]
3,470,71.0,2008-08-02 15:11:47,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,8,2008,"[web-services, .net]"
4,580,91.0,2008-08-02 23:30:59,NaN,21,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,8,2008,[sql-server]


In [0]:
# Let's see the Groups/Classes

tag_group = pd.read_csv(tag_group)
tag_group.head()

,name,group,nodesize
0,html,6,272.45
1,css,6,341.17
2,hibernate,8,29.83
3,spring,8,52.84
4,ruby,3,70.14


## 1. Text Pre-processing Pipeline

Every try-except block can be written as a different modular function which can be invoked from preprocess_text() function. This serves as a pipeline of the series of text-cleaning that you might require for your dataset.

In [0]:
clean = re.compile('<.*?>')

def preprocess_text(text) :
  try :
    # soup = BeautifulSoup(text, "html.parser")
    # text = soup.get_text()
    text=  re.sub(clean, '', text)
    text = html.unescape(text)
  except :
    print("Error in HTML Processing ...")
    print(text)
    text = text
  try :
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
  except :
    print("Error in removing extra lines ...")
    print(text)
    text = text

  try :
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
  except :
    print("Error in extra whitespace removal ...")
    print(text)
    text = text

  return text

## 2. Create Training Data Format

Here we iterate the dataset dataframe and create the format acceptable to Flair. This is a standard format for few other Text Classification models/frameworks by Facebook.

***Format***  : ____label ____**tag1** ____label ____**tag2** **text**

Here the text Document has to be in a single line which was handled in the preprocess_text() method.

In [0]:
def create_training_format(question_tag) :

  print("Preparing training data format ...")
  # training_df = pd.DataFrame("columns")
  labels = list()
  texts = list()
  for index in tqdm(question_tag.index) :
    tags = question_tag.loc[index,'Tag']
    text_label = ''
    for tag in tags :
      label = '__label__'+tag
      text_label = text_label + ' ' + label
    
    text_label = text_label.strip()
    # text =  html.unescape(question_tag.loc[index,'Body'])
    text =  question_tag.loc[index,'Title'].strip() + '. ' + question_tag.loc[index,'Body'].strip()

    # if len(text.split()) < 5 :
    #   continue 

    labels.append(text_label)
    texts.append(text)


  df = pd.DataFrame(list(zip(labels[:], texts[:])), columns =['label', 'text']) 
  # df.head()
  print("Cleaning Text ....")
  df['text'] = df['text'].apply(preprocess_text)
  print("Cleaned Data Size : {}".format(df.shape))

  return df






## 3. Create Training Splits

Here we create standard random splits of the dataset to :
  1. training set : 90 % data
  2. dev/validation set :  10 % data
  3. test set : 10 % data

#### TO DO : Experiments :
  1. Stratified Sampling
  2. Does a  70-15-15 split or 90-5-5 split make any difference when you ahve 1M records ?

In [0]:
def create_splits(df,path,group_id = ''):

  print("Splitting Training Data ... ")
  train_df , test_df = train_test_split(df,random_state=42,test_size=0.30)
  dev_df ,test_df = train_test_split(test_df,random_state=42,test_size=0.5)
  print("Training Dataset : {}".format(train_df.shape[0]))
  print("Validation Dataset : {}".format(dev_df.shape[0]))
  print("Test Dataset : {}".format(test_df.shape[0]))

  print("Path  : {} ".format(path+'training_data/group/'+ str(group_id) + '/train.txt'))
  train_df.to_csv(path+'training_data/group/'+ str(group_id) + '/train.txt',sep='\t',index=False,header=False)
  dev_df.to_csv(path+'training_data/group/'+ str(group_id) + '/dev.txt',sep='\t',index=False,header=False)
  test_df.to_csv(path+'training_data/group/'+ str(group_id) + '/test.txt',sep='\t',index=False,header=False)

  return train_df,dev_df,test_df


## 4. Divide the dataset into Groups/Classes

Here we iterate over the entire dataset to create group level datasets in the following steps :
  1. Iterate over the groups and read the full-dataset eveytime
  2. Get all the tags in the group from the **tag_group** lookup
  3. Iterate over training examples and see if the labels fall in the same group
  4. Remove training examples which don't belong to the group
  5. Create the training data format of the remaining dataset
  6. Split & Save the dataset

  ### TO DO :
    1. Make a single corpus for the entire dataset.


In [0]:
for grp_id in range(1,15):
  
  ## 1. Iterate over the groups and read the full-dataset eveytime
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp_id))
  print("=================================================================")
  print("Reading Pickle File ...")

  question_tag = pd.read_pickle(path+data)

  ## 2. Get all the tags in the group from the **tag_group** lookup  

  group =  tag_group[tag_group.group == grp_id]
  labels = list(set(group['name']))

  ## 3. Iterate over training examples and see if the labels fall in the same group
  for index in tqdm(question_tag.index):
    tags = question_tag.loc[index,'Tag']
    group_tags = list()
    for tag in tags :
      if tag in labels :
        group_tags.append(tag)
    question_tag.at[index,'Tag'] =  group_tags
  print("Before Removal of Blank Data : {} ".format(question_tag.shape))

   ## 4. Remove training examples which don't belong to the group
  question_tag = question_tag[question_tag['Tag'].map(lambda d: len(d)) > 0]
  print("Final Data for Group ID  : {} is {}".format(grp_id,question_tag.shape))

  ## 5. Create the training data format of the remaining dataset
  training_data_format = create_training_format(question_tag)

  ## 6. Split & Save the dataset
  train_df,dev_df,test_df = create_splits(training_data_format,path,group_id=grp_id)

  print("=================================================================")



Reading Pickle File ...


  0%|          | 1873/1051992 [00:00<00:56, 18726.22it/s]

Current Group ID : 2 


100%|██████████| 1051992/1051992 [00:37<00:00, 27773.86it/s]


Before Removal of Blank Data : (1051992, 10) 


...

Before Removal of Blank Data : (1051992, 10) 


 27%|██▋       | 3885/14291 [00:00<00:00, 38846.05it/s]

Final Data for Group ID  : 14 is (14291, 10)
Preparing training data format ...


100%|██████████| 14291/14291 [00:00<00:00, 36915.84it/s]


Cleaning Text ....
Cleaned Data Size : (14291, 2)
Splitting Training Data ... 
Training Dataset : 10003
Validation Dataset : 2144
Test Dataset : 2144
Path  : /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14/train.txt 


## 5. Create Corpus & Label Dictionary : Flair Corpus

For all the training splits created above for each group, we will be creating a corpus & vocabulary to train a different model.

In [0]:
path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/'

for grp in range(1,15):
  
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp))
  print("=================================================================")
  
  # this is the folder in which train, test and dev files reside
  data_folder =path+str(grp)+'/'
  print(data_folder)

  print("Creating Corpus ...")
  # load corpus containing training, test and dev data
  corpus: Corpus = ClassificationCorpus(data_folder,
                                        test_file='test.txt',
                                        dev_file='dev.txt',
                                        train_file='train.txt')

  # 2. create the label dictionary
  label_dict = corpus.make_label_dictionary()

  print("Obtaining Corpus Statisitics...")
  stats  = corpus.obtain_statistics()
  json_stats = json.loads(stats)

  print(json_stats)
  
  with open(data_folder+'corpus_statistics.json', 'w') as f:
    json.dump(json_stats, f)

  print("Creating Dumps ... ")
  with open(data_folder+'classification_corpus.pkl',mode='wb') as f :
    pickle.dump(corpus,f)

  with open(data_folder + 'classification_corpus_label_dict.pkl',mode='wb') as f:
    pickle.dump(label_dict,f)

/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/1/
Creating Corpus ...
2019-12-20 07:36:28,399 Reading data from /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/1
2019-12-20 07:36:28,402 Train: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/1/train.txt
2019-12-20 07:36:28,404 Dev: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/1/dev.txt
2019-12-20 07:36:28,406 Test: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/1/test.txt
2019-12-20 07:36:37,861 Computing label dictionary. Progress:


100%|██████████| 114121/114121 [05:25<00:00, 350.70it/s]

2019-12-20 07:42:03,422 [b'python', b'django', b'r', b'c', b'c++', b'matlab', b'qt', b'embedded', b'machine-learning', b'flask']
Creating Dumps ... 
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/2/
Creating Corpus ...
2019-12-20 07:42:03,438 Reading data from /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/2
2019-12-20 07:42:03,439 Train: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/2/train.txt
2019-12-20 07:42:03,440 Dev: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/2/dev.txt
2019-12-20 07:42:03,441 Test: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/2/test.txt


2019-12-20 07:42:12,136 Computing label dictionary. Progress:


100%|██████████| 137377/137377 [06:14<00:00, 366.66it/s]

...

100%|██████████| 13953/13953 [00:29<00:00, 471.11it/s]

2019-12-20 08:16:55,248 [b'regex', b'perl']
Creating Dumps ... 
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14/
Creating Corpus ...
2019-12-20 08:16:55,261 Reading data from /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14
2019-12-20 08:16:55,262 Train: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14/train.txt
2019-12-20 08:16:55,264 Dev: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14/dev.txt
2019-12-20 08:16:55,265 Test: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/group/14/test.txt


2019-12-20 08:17:01,750 Computing label dictionary. Progress:


100%|██████████| 10003/10003 [00:29<00:00, 335.86it/s]

2019-12-20 08:17:31,656 [b'excel', b'vba', b'excel-vba']
Creating Dumps ... 
